## Imports

In [61]:
# univariative
from metro_univariative2 import loadData as dataUni
from metro_univariative2 import splitData
from metro_univariative2 import rmse
from metro_univariative2 import to_supervised as to_supervised_uni
from metro_univariative2 import createModel as lstmUni
from metro_univariative2 import createModel2 as gruUni
from metro_univariative2 import createModel2 as cnnUni

# multivariative
from metro_multivariative import loadData as dataMulti
from keras.preprocessing.sequence import TimeseriesGenerator
# Outros
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import csv
from pickle import load

In [62]:
# for replicability purposes
tf.random.set_seed(91195003) 
np.random.seed(91190530) 
# for an easy reset backend session state 
tf.keras.backend.clear_session()

## Preparação geral (Univariante)

In [18]:
# carregar dados
data = dataUni()
# normalizar dados
scalertraffic = load(open('../models/scalerTraffic.pkl', 'rb'))
data[['traffic_volume']] = scalertraffic.transform(data[['traffic_volume']])
# dividir dados
train_df, val_df, test_df = splitData(data, 0.7, 0.2)

### Modelo Univariante 1 (LSTM)

In [9]:
# carregar parametros
# config = [neurónios, timesteps, batchsize, learning rate]
file_CSV = open('../models/uni_config_lstm.csv')
data_CSV = csv.reader(file_CSV)
config = list(data_CSV)
config = config[0]

# to supervised
testX, testy  = to_supervised_uni(val_df, int(config[1]))

# create model
model = tf.keras.models.load_model('../models/uni_lstm.h5', custom_objects={'rmse': rmse})
model.summary()
model.evaluate(testX,testy)

Model: "sequential_81"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_81 (LSTM)               (None, 32)                4352      
_________________________________________________________________
dense_162 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_163 (Dense)            (None, 1)                 33        
Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
_________________________________________________________________
278/278 [==============================] - 1s 2ms/step - loss: 0.0445 - mean_absolute_error: 0.0335 - rmse: 0.0444


[0.044457338750362396, 0.03350599855184555, 0.04441721737384796]

### Modelo Univariante 2 (GRU)

In [10]:
# carregar parametros
file_CSV = open('../models/uni_config_gru.csv')
data_CSV = csv.reader(file_CSV)
config = list(data_CSV)
config = config[0]

# to supervised
testX, testy  = to_supervised_uni(val_df, int(config[1]))

# create model
model = tf.keras.models.load_model('../models/uni_gru.h5', custom_objects={'rmse': rmse})
model.summary()
model.evaluate(testX,testy)

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_36 (GRU)                 (None, 32)                3360      
_________________________________________________________________
dense_72 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_73 (Dense)             (None, 1)                 33        
Total params: 4,449
Trainable params: 4,449
Non-trainable params: 0
_________________________________________________________________
278/278 [==============================] - 0s 2ms/step - loss: 0.0401 - mean_absolute_error: 0.0291 - rmse: 0.0401


[0.040088292211294174, 0.029069960117340088, 0.040066950023174286]

### Modelo Univariante 3 (CNN)

In [20]:
# carregar parametros
file_CSV = open('../models/uni_config_cnn.csv')
data_CSV = csv.reader(file_CSV)
config = list(data_CSV)
config = config[0]

# to supervised
testX, testy  = to_supervised_uni(val_df, int(config[1]))

# create model
model = tf.keras.models.load_model('../models/uni_cnn.h5', custom_objects={'rmse': rmse})
model.summary()
model.evaluate(testX,testy)

Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 24, 1)]           0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 20, 16)            96        
_________________________________________________________________
average_pooling1d_12 (Averag (None, 20, 8)             0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 160)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 16)                2576      
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 17        
Total params: 2,689
Trainable params: 2,689
Non-trainable params: 0
_______________________________________________________

[0.04632953926920891, 0.03353754058480263, 0.046320319175720215]

## Preparação geral (Multivariante)

In [63]:
# carregar dados
data = dataMulti()
# normalizar dados
scalerTemp = load(open('../models/scalerTemp.pkl', 'rb'))
data[['temp']] = scalertraffic.transform(data[['temp']])

scalerClouds = load(open('../models/scalerClouds.pkl', 'rb'))
data[['clouds_all']] = scalertraffic.transform(data[['clouds_all']])

scalertraffic = load(open('../models/scalerTraffic.pkl', 'rb'))
data[['traffic_volume']] = scalertraffic.transform(data[['traffic_volume']])
# dividir dados
train_df, val_df, test_df = splitData(data, 0.7, 0.2)

### Modelo 1 Multivariante (LSTM)

In [64]:
# carregar parametros
# config = [neurónios, timesteps, batchsize, learning rate]
file_CSV = open('../models/multi_config_lstm.csv')
data_CSV = csv.reader(file_CSV)
config = list(data_CSV)
config = config[0]

# to superviseddata1 = train_df
data1 = test_df
targets = data1[:,3]
dataTest = TimeseriesGenerator(data1, targets,
                        length=int(config[1]),
                        batch_size=int(config[2]))
# create model
model = tf.keras.models.load_model('../models/multi_lstm.h5', custom_objects={'rmse': rmse})
model.compile(
    loss = rmse, 
    optimizer = tf.keras.optimizers.Adam(learning_rate=float(config[3])), 
    metrics = ['mae',rmse])
model.summary()
model.evaluate(dataTest)

Model: "sequential_81"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_81 (LSTM)               (None, 16)                1408      
_________________________________________________________________
dense_162 (Dense)            (None, 16)                272       
_________________________________________________________________
dense_163 (Dense)            (None, 1)                 17        
Total params: 1,697
Trainable params: 1,697
Non-trainable params: 0
_________________________________________________________________
70/70 [==============================] - 0s 3ms/step - loss: 0.0448 - mae: 0.0344 - rmse: 0.0450


[0.04475574567914009, 0.034379661083221436, 0.04499175772070885]

### Modelo 2 Multivariante (GRU)

In [65]:
# carregar parametros
# config = [neurónios, timesteps, batchsize, learning rate]
file_CSV = open('../models/multi_config_gru.csv')
data_CSV = csv.reader(file_CSV)
config = list(data_CSV)
config = config[0]

# to superviseddata1 = train_df
data1 = test_df
targets = data1[:,3]
dataTest = TimeseriesGenerator(data1, targets,
                        length=int(config[1]),
                        batch_size=int(config[2]))
# create model
model = tf.keras.models.load_model('../models/multi_gru.h5', custom_objects={'rmse': rmse})
model.compile(
    loss = rmse, 
    optimizer = tf.keras.optimizers.Adam(learning_rate=float(config[3])), 
    metrics = ['mae',rmse])
model.summary()
model.evaluate(dataTest)

Model: "sequential_163"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_81 (GRU)                 (None, 64)                13632     
_________________________________________________________________
dense_326 (Dense)            (None, 64)                4160      
_________________________________________________________________
dense_327 (Dense)            (None, 1)                 65        
Total params: 17,857
Trainable params: 17,857
Non-trainable params: 0
_________________________________________________________________
70/70 [==============================] - 0s 3ms/step - loss: 0.0779 - mae: 0.0652 - rmse: 0.0777


[0.07789655774831772, 0.06519317626953125, 0.07770460844039917]

### Modelo 3 Multivariante (CNN)

In [66]:
# carregar parametros
# config = [neurónios, timesteps, batchsize, learning rate]
file_CSV = open('../models/multi_config_cnn.csv')
data_CSV = csv.reader(file_CSV)
config = list(data_CSV)
config = config[0]

# to superviseddata1 = train_df
data1 = test_df
targets = data1[:,3]
dataTest = TimeseriesGenerator(data1, targets,
                        length=int(config[1]))
# create model
model = tf.keras.models.load_model('../models/multi_cnn.h5', custom_objects={'rmse': rmse})
model.summary()
model.evaluate(dataTest)

Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24, 5)]           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 16)            416       
_________________________________________________________________
average_pooling1d_1 (Average (None, 20, 8)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2576      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0
_______________________________________________________

[0.07041484117507935, 0.05430785194039345, 0.07040555030107498]